# 1. Configuration

In [1]:
import pandas as pd
import numpy as np
import gc
import time
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os
import joblib
import random
import math
from tqdm.auto import tqdm 

from scipy.interpolate import interp1d

from math import pi, sqrt, exp
import sklearn,sklearn.model_selection
import torch
from torch import nn,Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler
plt.style.use("ggplot")

from pyarrow.parquet import ParquetFile
import pyarrow as pa 
import ctypes

def normalize(y):
    mean = y[:,0].mean().item()
    std = y[:,0].std().item()
    y[:,0] = (y[:,0]-mean)/(std+1e-16)
    mean = y[:,1].mean().item()
    std = y[:,1].std().item()
    y[:,1] = (y[:,1]-mean)/(std+1e-16)
    return y

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
EPOCHS = 5
WARMUP_PROP = 0.2
BS = 16
WORKERS = 4
TRAIN_PROP = 0.9
max_chunk_size = 150000
if device=='cpu':
    torch.set_num_interop_threads(WORKERS)
    torch.set_num_threads(WORKERS)

In [3]:
def plot_history(history, model_path=".", show=True):
    epochs = range(1, len(history["train_loss"]) + 1)

    plt.figure()
    plt.plot(epochs, history["train_loss"], label="Training Loss")
    plt.plot(epochs, history["valid_loss"], label="Validation Loss")
    plt.title("Loss evolution")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.savefig(os.path.join(model_path, "loss_evo.png"))
    if show:
        plt.show()
    plt.close()

#     plt.figure()
#     plt.plot(epochs, history["valid_mAP"])
#     plt.title("Validation mAP evolution")
#     plt.xlabel("Epochs")
#     plt.ylabel("mAP")
#     plt.savefig(os.path.join(model_path, "mAP_evo.png"))
#     if show:
#         plt.show()
#     plt.close()

    plt.figure()
    plt.plot(epochs, history["lr"])
    plt.title("Learning Rate evolution")
    plt.xlabel("Epochs")
    plt.ylabel("LR")
    plt.savefig(os.path.join(model_path, "lr_evo.png"))
    if show:
        plt.show()
    plt.close()

# Define Dataset

In [4]:
# SIGMA = 720 #average length of day is 24*60*12 = 17280 for comparison
# SAMPLE_FREQ = 12 # 1 obs per minute
# class SleepDataset(Dataset):
#     def __init__(
#         self,
#         file
#     ):
#         self.targets,self.data,self.ids = joblib.load(file)
            
#     def downsample_seq_generate_features(self,feat, downsample_factor = SAMPLE_FREQ):
#         # downsample data and generate features
#         if len(feat)%SAMPLE_FREQ!=0:
#             feat = np.concatenate([feat,np.zeros(SAMPLE_FREQ-((len(feat))%SAMPLE_FREQ))+feat[-1]])
#         feat = np.reshape(feat, (-1,SAMPLE_FREQ))
#         feat_mean = np.mean(feat,1)
#         feat_std = np.std(feat,1)
#         feat_median = np.median(feat,1)
#         feat_max = np.max(feat,1)
#         feat_min = np.min(feat,1)

#         return np.dstack([feat_mean,feat_std,feat_median,feat_max,feat_min])[0]
#     def downsample_seq(self,feat, downsample_factor = SAMPLE_FREQ):
#         # downsample data
#         if len(feat)%SAMPLE_FREQ!=0:
#             feat = np.concatenate([feat,np.zeros(SAMPLE_FREQ-((len(feat))%SAMPLE_FREQ))+feat[-1]])
#         feat = np.reshape(feat, (-1,SAMPLE_FREQ))
#         feat_mean = np.mean(feat,1)
#         return feat_mean
    
#     def gauss(self,n=SIGMA,sigma=SIGMA*0.15):
#         # guassian distribution function
#         r = range(-int(n/2),int(n/2)+1)
#         return [1 / (sigma * sqrt(2*pi)) * exp(-float(x)**2/(2*sigma**2)) for x in r]
    
#     def __len__(self):
#         return len(self.targets)

#     def __getitem__(self, index):
#         X = self.data[index][['anglez','enmo']]
#         y = self.targets[index]

#         # turn target inds into array
#         target_guassian = np.zeros((len(X),2))
#         for s,e in y:
#             st1,st2 = max(0,s-SIGMA//2),s+SIGMA//2+1
#             ed1,ed2 = e-SIGMA//2,min(len(X),e+SIGMA//2+1)
#             target_guassian[st1:st2,0] = self.gauss()[st1-(s-SIGMA//2):]
#             target_guassian[ed1:ed2,1] = self.gauss()[:SIGMA+1-((e+SIGMA//2+1)-ed2)]
#             gc.collect()
#         y = target_guassian
#         gc.collect()
#         X = np.concatenate([self.downsample_seq_generate_features(X.values[:,i],SAMPLE_FREQ) for i in range(X.shape[1])],-1)
#         gc.collect()
#         y = np.dstack([self.downsample_seq(y[:,i],SAMPLE_FREQ) for i in range(y.shape[1])])[0]
#         gc.collect()
#         y = normalize(y)
#         #y = normalize(torch.from_numpy(y))
#         #X = torch.from_numpy(X)
        
#         result = np.concatenate((X, y), axis=1)
        
#         return result
    
    
# train_ds = SleepDataset('/kaggle/input/sleep-critical-point-prepare-data/train_data.pkl')

In [5]:
def time_slide(train_ds, window_size, forcast_size):
    data_list = []
    dap_list = []
    
    for i in tqdm(range(len(train_ds))):
        X = train_ds[i]

        for idx in range(0, len(X)-window_size-forcast_size):
            data_list.append(X[idx : idx+window_size])
            dap_list.append(X[idx+window_size : idx+window_size+forcast_size, -2:])
            
    return np.array(data_list, dtype='float16'), np.array(dap_list, dtype='float16')

In [6]:
results = time_slide(train_ds, 30, 10)

In [7]:
import joblib

joblib.dump((results), 'time_slide_data.pkl')

# Data load

In [8]:
results = joblib.load('/kaggle/input/time-slide/time_slide_data.pkl')

In [9]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [10]:
class moving_avg(torch.nn.Module):
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = torch.nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

class series_decomp(torch.nn.Module):
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        residual = x - moving_mean
        return moving_mean, residual 
    
class LTSF_DLinear(torch.nn.Module):
    def __init__(self, window_size, forcast_size, kernel_size, individual, feature_size):
        super(LTSF_DLinear, self).__init__()
        self.window_size = window_size
        self.forcast_size = forcast_size
        self.decompsition = series_decomp(kernel_size)
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear_Seasonal = torch.nn.ModuleList()
            self.Linear_Trend = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Trend.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Trend[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
                self.Linear_Seasonal.append(torch.nn.Linear(self.window_size, self.forcast_size))
                self.Linear_Seasonal[i].weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
        else:
            self.Linear_Trend = torch.nn.Linear(self.window_size, self.forcast_size)
            self.Linear_Trend.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))
            self.Linear_Seasonal = torch.nn.Linear(self.window_size,  self.forcast_size)
            self.Linear_Seasonal.weight = torch.nn.Parameter((1/self.window_size)*torch.ones([self.forcast_size, self.window_size]))

    def forward(self, x):
        
        trend_init, seasonal_init = self.decompsition(x)
        trend_init, seasonal_init = trend_init.permute(0,2,1), seasonal_init.permute(0,2,1)
        if self.individual:
            trend_output = torch.zeros([trend_init.size(0), trend_init.size(1), self.forcast_size], dtype=trend_init.dtype).to(trend_init.device)
            seasonal_output = torch.zeros([seasonal_init.size(0), seasonal_init.size(1), self.forcast_size], dtype=seasonal_init.dtype).to(seasonal_init.device)
            for idx in range(self.channels):
                trend_output[:, idx, :] = self.Linear_Trend[idx](trend_init[:, idx, :])
                seasonal_output[:, idx, :] = self.Linear_Seasonal[idx](seasonal_init[:, idx, :])                
        else:
            trend_output = self.Linear_Trend(trend_init)
            seasonal_output = self.Linear_Seasonal(seasonal_init)
        x = seasonal_output + trend_output
        x = x.permute(0,2,1)
        x = x[:, :, -2:]
        
        return x

# Train and Eval

In [11]:
def train(model, train_loader, val_loader, device, scheduler):
    #model = nn.DataParallel(model, device_ids=[0, 1], output_device=0)
    model.to(device)

    criterion = torch.nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    best_score = 0
    max_loss = np.inf
    best_model = None
    
    train_loss_list = []
    valid_loss_list = []
    
    for epoch in range(1, 5):
        model.train()
        train_loss = []
        valid_loss = []
        for X, Y in tqdm(iter(train_loader)):
            optimizer.zero_grad()
            X, Y = X.to(device), Y.to(device)
            output = model(X)
            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        train_loss_list.append(np.mean(train_loss))
        
        model.eval()
        with torch.no_grad():
            for X, Y in tqdm(iter(val_loader)):
                X, Y = X.to(device), Y.to(device)
                output = model(X)    
                loss = criterion(output, Y)
                valid_loss.append(loss.item())
                
        valid_loss_list.append(np.mean(valid_loss))
        
        if scheduler is not None:
            scheduler.step()

        if loss < max_loss:
            torch.save(model, './DLinear_model.pth')
            max_loss = loss
            
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{np.mean(valid_loss):.5f}]')
    
    return model, train_loss_list, valid_loss_list

In [12]:
train_input = results[0]
train_target = results[1]
del results

In [13]:
train_input_ids = train_input[:, 0, 0].astype(int)

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [14]:
train_loss_list = []
valid_loss_list = []

In [15]:
for fold_num, (train_idx, valid_idx) in enumerate(skf.split(train_input, train_input_ids)):
    print(f'fold: {fold_num}', '='*50)
    
    train_input_data, train_target_data = train_input[train_idx], train_target[train_idx]
    val_input_data, val_target_data = train_input[valid_idx], train_target[valid_idx]

    train_dataset = CustomDataset(train_input_data, train_target_data)
    train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=False, num_workers=16, pin_memory=True)

    val_dataset = CustomDataset(val_input_data, val_target_data)
    val_loader = DataLoader(val_dataset, batch_size = 16, shuffle=False, num_workers=16, pin_memory=True)

    model = LTSF_DLinear(window_size=30, forcast_size=10, kernel_size=25, individual=False, feature_size=10)

    scheduler = None

    print("Start Training")
    infer_model, train_loss_list, valid_loss_list = train(model, train_loader, val_loader, device, scheduler)
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_loss_list, label='Training Loss')
    plt.plot(valid_loss_list, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.show()
    
    image_file_path = f'./losses_plot_fold_{fold_num}.png'
    plt.savefig(image_file_path)

    del train_input_data, train_target_data, val_input, val_target, train_dataset, train_loader
    
    break

fold: 0 ==================================================


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Start Training


  0%|          | 0/599126 [00:00<?, ?it/s]

  0%|          | 0/66570 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [nan] Val Loss : [nan]


  0%|          | 0/599126 [00:00<?, ?it/s]

  0%|          | 0/66570 [00:00<?, ?it/s]

Epoch : [2] Train Loss : [nan] Val Loss : [nan]


  0%|          | 0/599126 [00:00<?, ?it/s]